In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.NRMS import NRMSModel

In [6]:
hparams = {
    'scale':'demo',
    'name':'nrms',
    'batch_size':5,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'query_dim':200,
    'embedding_dim':300,
    'value_dim':16,
    'head_num':16,
    'epochs':1,
    'metrics':'auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:0',
    'attrs': ['title'],
    'k':-1,
    'select':None,
    'save_step':[0],
    'save_each_epoch':False,
    'train_embedding':False,
    'mode':'train',
    'news_id':False,
    'validate':False,
}

In [3]:

vocab, loaders = prepare(hparams)

In [8]:
nrmsModel = NRMSModel(vocab=vocab,hparams=hparams).to(hparams['device'])

In [7]:
train(nrmsModel, hparams, loaders, interval=10)

training...
0it [00:00, ?it/s]


RuntimeError: Found dtype Long but expected Float